In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import requests
import random
import yfinance as yf

### 7.2.2 Reading & Writing Files in Python

*Note: If you are not running this notebook from the default GitHub directory, please download `AAPL.csv` from the `data/` folder, upload it to your current working directory, and remove `data/` from the file path.*

In [ ]:
# read csv
df = pd.read_csv('data/AAPL.csv', index_col='Date')

In [ ]:
# open csv line by line
# this cell is for demonstration purposes only, please do not execute it

f = open('AAPL.csv', 'r')
line = f.readline()
while line:
	parse_line()              # placeholder for processing the line read from the file
	line = f.readline()
f.close()

### 7.2.3 Parsing Data from a Website

*Note: Wikipedia pages referenced in the book are no longer reliably accessible via `pd.read_html()` and `requests.get()`, so this notebook uses the UIUC Graybook salary tables instead for demonstration purposes.*

In [ ]:
# read html into a dataframe
url = 'https://discovery.cs.illinois.edu/guides/Reading-Data/pandas-read-html/'
dfs = pd.read_html(url, index_col=0)
dfs[0].head()

,Employee Name,Job Title,Tenure,Empl Class,Present FTE,Proposed FTE,Present Salary,Proposed Salary
0,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences,KV - Liberal Arts & Sciences
1,303 - African American Studies,303 - African American Studies,303 - African American Studies,303 - African American Studies,303 - African American Studies,303 - African American Studies,303 - African American Studies,303 - African American Studies
2,"Anderson, James D",PROF,NaN,AA,0.00,0.00,$0.00,$0.00
3,"Anderson, James D",Employee Total for All Jobs...,Employee Total for All Jobs...,Employee Total for All Jobs...,1.00,1.00,"$319,633.33","$325,797.33"
4,"Bailey, Ronald William",HEAD,NaN,BC,0.00,0.00,"$4,400.00","$4,400.00"


In [ ]:
# read html into a long string
response = requests.get(url)
print(response.text[:500])

<!doctype html><html lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>Creating a DataFrame from an HTML table using Pandas - Data Science Discovery</title><meta name="description" content="HTML tables can be found on many different websites and can contain useful data we may want to analyze."><link rel="icon" type="image/svg+xml" href="/static/discovery.svg"><link rel="canonical" href="https://discovery.cs.illinois.edu/guides/Reading


### 7.2.4 Interacting with Databases in Python

In [ ]:
# this cell is for explanation purposes only
# readers will need to specify the server, database, and user credentials

!pip install pypyodbc
import pypyodbc as pyodbc

connection_string = f'Driver={SQL Server};Server={server_name};Database={db_name};UID={user_id};PWD={password};'
db = pyodbc.connect(connection_string)
qry = 'select * from table'
df = pd.read_sql_query(qry, db)

In [ ]:
# fetch data from Yahoo! finance database
df = yf.download(['AAPL'], start='2024-01-01', end='2024-01-31', auto_adjust=False)
df.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2024-01-02,183.903229,185.639999,188.440002,183.889999,187.149994,82488700
2024-01-03,182.526230,184.250000,185.880005,183.429993,184.220001,58414500
2024-01-04,180.208130,181.910004,183.089996,180.880005,182.149994,71983600
2024-01-05,179.484940,181.179993,182.759995,180.169998,181.990005,62379700
2024-01-08,183.823990,185.559998,185.600006,181.500000,182.089996,59144500


Please note that [`yfinance`](https://github.com/ranaroussi/yfinance) is not an official tool offered by [Yahoo!Ⓡ finance](https://finance.yahoo.com/). It works well for fetching financial & market data from [Yahoo!Ⓡ finance](https://finance.yahoo.com/) as of December 2025 but may change without notice. Please check their GitHub page for further information.

### 7.6.1 Interpolation & Filling Forward

The following example shows how we can use the functionality embedded in Python's Data Frames to fill missing data via interpolation or filling forward:

In [ ]:
df = pd.DataFrame(data={'date': pd.date_range(start='2021-2-19', end='2021-2-26'),
	                      'price': [78, 73, np.nan, 75, 79, 83, np.nan, 78]})
df

,date,price
0,2021-02-19,78.0
1,2021-02-20,73.0
2,2021-02-21,NaN
3,2021-02-22,75.0
4,2021-02-23,79.0
5,2021-02-24,83.0
6,2021-02-25,NaN
7,2021-02-26,78.0


In [ ]:
# interpolation
df_inter = df.copy()
df_inter.loc[:, 'price'] = df_inter.loc[:, 'price'].interpolate(method='linear')
df_inter

,date,price
0,2021-02-19,78.0
1,2021-02-20,73.0
2,2021-02-21,74.0
3,2021-02-22,75.0
4,2021-02-23,79.0
5,2021-02-24,83.0
6,2021-02-25,80.5
7,2021-02-26,78.0


In [ ]:
# forward fill
df_forward = df.copy()
df_forward.ffill()

,date,price
0,2021-02-19,78.0
1,2021-02-20,73.0
2,2021-02-21,73.0
3,2021-02-22,75.0
4,2021-02-23,79.0
5,2021-02-24,83.0
6,2021-02-25,83.0
7,2021-02-26,78.0


### 7.6.2 Filling via Regression

In this coding example, we show how regression techniques can be used to fill in (hypothetically) missing data for the XLK technology ETF based on market returns in the S&P 500 ETF, SPY.  In this exercise we consider the case of having historical data from March 1, 2019 to February 29, 2024, but also suppose that all entries for XLK in January 2024 are missing.  As we detailed in the prior section, we fit a linear regression model to estimate the coefficient, or beta of XLK to the S&P 500.  It should again be emphasized that this approach assumes a significant amout of co-movement, or correlation, in the returns in the two ETFs to be successful.  Finally, once we have calculate the parameter, $\beta$, then we used this beta to infer the returns of XLK for all days in January 2024. This is detailed in the code below:

In [ ]:
df = yf.download(['XLK', 'SPY'], start='2019-03-01', end='2024-02-29', auto_adjust=False)['Adj Close']
df = df.pct_change().dropna()
df_missing = df['2024-01-01':'2024-01-31']
df_missing.head()

[*********************100%***********************]  2 of 2 completed


Ticker,SPY,XLK
Date,,
2024-01-02,-0.005596,-0.026185
2024-01-03,-0.008167,-0.010190
2024-01-04,-0.003221,-0.007330
2024-01-05,0.001370,-0.000272
2024-01-08,0.014276,0.025092


In [ ]:
# For backtesting or prediction, only historical data is used
df_boot1 = df[df.index.min():'2023-12-31']
linear_reg1 = sm.OLS(df_boot1['XLK'], df_boot1['SPY']).fit()
beta_hat1 = float(linear_reg1.params.values[0])
xlk_boot1 = df_missing['SPY'] * beta_hat1
xlk_boot1.head()

,SPY
Date,
2024-01-02,-0.006837
2024-01-03,-0.009978
2024-01-04,-0.003935
2024-01-05,0.001673
2024-01-08,0.017441


In [ ]:
# For risk management purposes, the whole time period can be used (except the missing period)
df_boot2 = pd.concat([df_boot1, df['2024-02-01':'2024-02-29']])
linear_reg2 = sm.OLS(df_boot2['XLK'], df_boot2['SPY']).fit()
beta_hat2 = float(linear_reg2.params.values[0])
xlk_boot2 = df_missing['SPY'] * beta_hat2
xlk_boot2.head()

,SPY
Date,
2024-01-02,-0.006838
2024-01-03,-0.009979
2024-01-04,-0.003936
2024-01-05,0.001674
2024-01-08,0.017444


The reader should notice that for different applications of this technique, different time periods may be used to calculate the $\beta$.  When doing backtesting or making predictions, it is important to use historical data only because we do not want to include any information in the future.  That is, we want to ensure that we are not looking ahead.  For risk management purposes, however, it may be better to use all available data.  This is because risk management calculations are less sensitive to look-ahead bias, and the additional data can help us recover a more stable estimate of the coefficient $\beta$.

### 7.6.3 Filling via Bootstrapping

In this coding example, we use the historical data of SPY, AAPL, GOOG to show how to generate a paths of synthetic returns via bootstrapping techniques:

In [ ]:
df_price = yf.download(['SPY', 'AAPL', 'GOOG'], start='2020-03-01', end='2024-02-29', auto_adjust=False)['Adj Close']
df_ret = df_price.pct_change().dropna()
N = df_ret.shape[0]
m = 10    # length of the missing data period
boot_index = random.sample(range(N), m)
df_boot = df_ret.iloc[boot_index].set_index(pd.date_range(start='2024-03-01', end='2024-03-14', freq='B'))
df_boot

[*********************100%***********************]  3 of 3 completed


Ticker,AAPL,GOOG,SPY
2024-03-01,0.050503,0.042437,0.029493
2024-03-04,-0.019246,-0.011560,-0.000773
2024-03-05,-0.035787,-0.009721,-0.005596
2024-03-06,0.002053,-0.001855,-0.002328
2024-03-07,0.000487,-0.001693,-0.005188
2024-03-08,-0.058680,-0.058639,-0.043482
2024-03-11,-0.012692,-0.004536,-0.000335
2024-03-12,0.025595,0.020034,0.033568
2024-03-13,0.010139,0.053412,0.007630
2024-03-14,0.004175,0.002933,0.005396


As a note, it is theoretically possible that the number of missing datapoints, or the length of our bootstrapped paths could exceed the length of the historical dataset. These situations may lead to a great deal of sampling however, as the lack of data makes it hard to infer the empirical distribution from the observations. As a result, bootstrapping may not work well in these situations. Instead, it will be much more effective when the historical dataset is large, increasing our confidence in our estimates of the empirical distribution.